# Converting Images to Region Adjacency Graphs (RAGs)
---

## Dependencies

1. **pickle**
  - Import RAG objects
2. **numpy**
  - For calculating L1 distance between two vectors


In [15]:
import pickle
import numpy as np

---

#### Importing graph objects created earlier

In [4]:
graphs = pickle.load(open("../outputs/pickle/graphs.file", 'rb'))
print(f"{len(graphs)} loaded from file.")

1280 loaded from file.


---

In [5]:
def get_max_degree_node(graph):
    max_degree = 0
    max_degree_node = None
    
    for degree in graph.degree:
        if degree[1] > max_degree:
            max_degree = degree[1]
            max_degree_node = degree[0]
    
    return max_degree_node

In [12]:
error_graphs = []
def priority_bfs(graph, root):
    if root is None:
        return [(1,40000)]
    vector  = []
    visited = []
    queue   = []
    
    visited.append(root)
    queue.append(root)
    
    #set_trace()
    
    while queue:
        current_node = queue.pop(0)
        try:
            vector.append((current_node, graph.nodes()[current_node]['area']))
        except Exception as e:
            print("xxxxxxxxxxxxxxxxxxxxx")
            print(root)
            print(e)
            error_graphs.append(graph)
            return ""
            #nx.draw_networkx(graph)
        
        # populating neigbors of the current node 
        # and sorting them based on size

        current_node_neighbors = []
        current_node_neighbors_with_size = []
        for neighbor in graph.neighbors(current_node):
            current_node_neighbors_with_size.append((neighbor, graph.nodes()[current_node]['area']))
        current_node_neighbors_with_size.sort(key = lambda x: x[1])
        for i in current_node_neighbors_with_size:
            current_node_neighbors.append(i[0])
            
            
        for neighbor in current_node_neighbors:
            if neighbor not in visited:
                visited.append(neighbor)
                queue.append(neighbor)
                
    return vector      
        

In [13]:
vectors = []
for graph in graphs:
    # set the node with the highest edges as root
    root = get_max_degree_node(graph)
    
    # get BFS vector
    vector = priority_bfs(graph, root)
    vectors.append(vector)

In [16]:
# normalize dimensions of all vectores
max_dimensions = max((map(len, vectors)))

new_vectors = []
for i in vectors:
    vector = [j[1] for j in i]
    vector += [0]*(max_dimensions-len(i))
    new_vectors.append(np.array(vector))

In [17]:
D = []
for vector_1 in new_vectors:
    D_row = []
    for vector_2 in new_vectors:
        diff = np.linalg.norm((vector_1 - vector_2), ord=1)
        D_row.append(diff)
    D.append(D_row)

In [19]:
a = np.asarray(D)
np.savetxt("../outputs/distance/bfs_D.csv", a, delimiter=",")

#### Creating smaller dataset to test with Entropy-Isomap

(first 30 points of the first 2 trajectories)

In [63]:
small_data = []
small_data += new_vectors[:30]
small_data += new_vectors[80:110]

D_small_data = []
for vector_1 in small_data:
    D_row = []
    for vector_2 in small_data:
        diff = np.linalg.norm((vector_1 - vector_2), ord=1)
        D_row.append(diff)
    D_small_data.append(D_row)

a = np.asarray(D_small_data)
np.savetxt("bfs_D_small_data.csv", a, delimiter=",")